In [1]:
import pywren, json, requests, time, bs4
import pandas as pd
from re import findall
import numpy as np

In [2]:
data = pd.read_csv('expert_info_v1.csv')
mobile = "https://mobile.twitter.com"
tar_accounts = list(set(data['twitter'])
print(len(tar_accounts))

685


In [62]:
## for each page, extract followers/ following
def Follow(response):
    soup = bs4.BeautifulSoup(response, "html.parser")
    follow = soup.find_all("td", "info fifty screenname")
    cursor = soup.find_all("div", "w-button-more")
    try: cursor = findall(r'cursor=(.*?)">', str(cursor))[0]
    except IndexError: pass  
    follow = [i.find('a')['name'] for i in follow]
    return follow, cursor

## given a user, loop through its followers/ following pages using cursors
def scrape_fols(username, fol):
    mobile = "https://mobile.twitter.com"
    url = f"{mobile}/{username}/{fol}?lang=en"
    init = '-1'
    follower_lst = []
    count = 0
    
    while(len(init)!=0):
        if init == '-1': tar_url = url
        else: tar_url = url + f"&cursor={init}"
        r = requests.get(tar_url)
        
        for i in range(5):
            if r: break
            else: 
                time.sleep(3)
                r = requests.get(tar_url).text
                
        if not r: break
        
        followers, init = Follow(r.text)
        follower_lst.extend(followers)
        print(len(follower_lst), end = '\r')
    return follower_lst

def scrape_user(username):
    expert = dict()
    expert["followers"] = scrape_fols(username, fol = 'followers')
    expert["following"] = scrape_fols(username, fol = 'following')
    return {username: expert}

In [38]:
pwex = pywren.default_executor(job_max_runtime = 500)
futures = pwex.map(scrape_user, tar_accounts)
print(len(futures))

685


In [51]:
dones, not_dones = pywren.wait(futures, pywren.ANY_COMPLETED)
print(len(dones), len(not_dones), end = '\r')

In [52]:
expert_data = dict()
for num, item in enumerate(dones):
    try: expert_data.update(item.result())
    except (IndexError, AttributeError): continue

In [54]:
extra = set(tar_accounts) - expert_data.keys()
len(extra)

71

In [ ]:
extra = set(tar_accounts) - expert_data.keys()
for user in extra:
    print(user)
    add = scrape_user(user)
    expert_data.update(add)
    print(len(expert_data[user]['followers']), 
          len(expert_data[user]['following']))
    json.dump(expert_data, open('expert_network.json', 'w'))

JuliaGillard
